### Installs

In [1]:
!pip install simpletransformers

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import re
import numpy as np
import sklearn

### Data

In [4]:
train = pd.read_csv('/content/drive/My Drive/1_Studium/2_Master/3_Sem/Team_Project/Data/small_train.csv')
test = pd.read_csv('/content/drive/My Drive/1_Studium/2_Master/3_Sem/Team_Project/Data/test_tables_seq.csv')

In [5]:
train

,Unnamed: 0,text,name,class
0,0,Bierwurst.com Cupids Dating Tips.com Ramsbury....,name,Product
1,1,"{'price': '12950', 'availability': 'https://sc...",offers,Product
2,2,Food and Drink Lifestyle > Dating Travel Desig...,category,Product
3,3,2020-09-16T02:57:04+01:00 2020-09-10T08:33:11+...,releasedate,Product
4,4,"[{'name': 'pa_internet-archive-age', 'value': ...",additionalproperty,Product
...,...,...,...,...
11814,11814,"{'ratingvalue': '4.63', 'ratingcount': '8'},{'...",aggregaterating,Recipe
11815,11815,Baked Keto mascarpone cheesecake Keto red curr...,keywords,Recipe
11816,11816,"{'saturatedfatcontent': '17 g', 'servingsize':...",nutrition,Recipe
11817,11817,{'description': 'A delicious low carb keto bak...,mainentityofpage,Recipe


In [6]:
train["label_complete"] = train["name"] + train["class"]
test["label_complete"] = test["name"] + test["class"]

In [7]:
train.shape[0]

11819

In [8]:
train[(train['name']=='inalbum') &(train['class']=='MusicRecording')].index

Int64Index([  119,   202,   225,   262,   336,   350,   369,   373,   468,
              527,
            ...
            11201, 11294, 11318, 11352, 11366, 11429, 11465, 11710, 11714,
            11762],
           dtype='int64', length=198)

In [9]:
train.shape

(11819, 5)

In [10]:
#MusicRecording: inalbum
#Event: performer
#Event: eventattendancemode
#Event: eventstatus

In [11]:
train.drop(train[(train['name']=='inalbum') &(train['class']=='MusicRecording')].index, inplace=True)

In [12]:
train.drop(train[(train['name']=='performer') &(train['class']=='Event')].index, inplace=True)

In [13]:
train.drop(train[(train['name']=='eventattendancemode') &(train['class']=='Event')].index, inplace=True)

In [14]:
train.drop(train[(train['name']=='eventstatus') &(train['class']=='Event')].index, inplace=True)

In [15]:
train.drop(['Unnamed: 0', 'class'], axis=1, inplace=True)
test.drop(['Unnamed: 0', 'class'], axis=1, inplace=True)

In [16]:
train['text'] = train.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))


In [17]:
num_columns = len(set.union(set(test.label_complete.unique()), set(train.label_complete.unique())))

In [18]:
set(test.label_complete.unique()).difference( set(train.label_complete.unique()))

{'founderLocalBusiness'}

In [20]:
labels_new = {'worstratingProduct': 0,
 'acceptsreservationsRestaurant': 1,
 'additionalnamePerson': 2,
 'additionalpropertyProduct': 3,
 'addressHotel': 4,
 'addressLocalBusiness': 5,
 'addressPerson': 6,
 'addressProduct': 7,
 'addressRestaurant': 8,
 'addresslocalityLocalBusiness': 9,
 'addressregionLocalBusiness': 10,
 'affiliationPerson': 11,
 'aggregateratingBook': 12,
 'aggregateratingCreativeWork': 13,
 'aggregateratingHotel': 14,
 'aggregateratingLocalBusiness': 15,
 'aggregateratingProduct': 16,
 'aggregateratingRecipe': 17,
 'aggregateratingRestaurant': 18,
 'alternatenameProduct': 19,
 'alternativeheadlineCreativeWork': 20,
 'areaservedLocalBusiness': 21,
 'articlebodyCreativeWork': 22,
 'audienceProduct': 23,
 'authorBook': 24,
 'authorCreativeWork': 25,
 'authorProduct': 26,
 'authorRecipe': 27,
 'availabilityProduct': 28,
 'bestratingProduct': 29,
 'birthdatePerson': 30,
 'birthplacePerson': 31,
 'bookeditionBook': 32,
 'bookformatBook': 33,
 'brandProduct': 34,
 'breadcrumbProduct': 35,
 'byartistMusicAlbum': 36,
 'byartistMusicRecording': 37,
 'categoryProduct': 38,
 'citystatezipLocalBusiness': 39,
 'colorProduct': 40,
 'commentcountCreativeWork': 41,
 'conditionProduct': 42,
 'contactpointLocalBusiness': 43,
 'contactpointPerson': 44,
 'cookingmethodRecipe': 45,
 'cooktimeRecipe': 46,
 'copyrightholderCreativeWork': 47,
 'copyrightyearCreativeWork': 48,
 'creatorCreativeWork': 49,
 'datecreatedCreativeWork': 50,
 'datecreatedMusicRecording': 51,
 'datemodifiedCreativeWork': 52,
 'datemodifiedRecipe': 53,
 'datepublishedBook': 54,
 'datepublishedCreativeWork': 55,
 'datepublishedMusicRecording': 56,
 'datepublishedProduct': 57,
 'datepublishedRecipe': 58,
 'deathdatePerson': 59,
 'depthProduct': 60,
 'disambiguatingdescriptionProduct': 61,
 'doortimeEvent': 62,
 'durationEvent': 63,
 'durationMusicRecording': 64,
 'emailHotel': 65,
 'emailLocalBusiness': 66,
 'emailPerson': 67,
 'enddateEvent': 68,
 'episodenumberTVEpisode': 69,
 'worksforPerson': 70,
 'familynamePerson': 71,
 'faxnumberLocalBusiness': 72,
 'faxnumberPerson': 73,
 'genderPerson': 74,
 'genreBook': 75,
 'genreCreativeWork': 76,
 'genreMusicRecording': 77,
 'geoHotel': 78,
 'geoLocalBusiness': 79,
 'geoPlace': 80,
 'geoProduct': 81,
 'geoRestaurant': 82,
 'givennamePerson': 83,
 'gtin12Product': 84,
 'gtin13Product': 85,
 'gtin14Product': 86,
 'gtin8Product': 87,
 'gtinProduct': 88,
 'hasmapLocalBusiness': 89,
 'hasmenuRestaurant': 90,
 'headlineCreativeWork': 91,
 'headlineRecipe': 92,
 'heightPerson': 93,
 'heightProduct': 94,
 'homelocationPerson': 95,
 'identifierProduct': 96,
 'worklocationPerson': 97,
 'ingredientsRecipe': 98,
 'inlanguageBook': 99,
 'inlanguageCreativeWork': 100,
 'interactioncountCreativeWork': 101,
 'interactionstatisticCreativeWork': 102,
 'interactiontypeCreativeWork': 103,
 'isbnBook': 104,
 'ispartofRecipe': 105,
 'isrelatedtoProduct': 106,
 'issimilartoProduct': 107,
 'itemconditionProduct': 108,
 'itemlistelementProduct': 109,
 'jobtitlePerson': 110,
 'keywordsCreativeWork': 111,
 'keywordsRecipe': 112,
 'knowslanguagePerson': 113,
 'legalnameLocalBusiness': 114,
 'locationEvent': 115,
 'locationLocalBusiness': 116,
 'mainentityofpageCreativeWork': 117,
 'mainentityofpagePerson': 118,
 'mainentityofpageProduct': 119,
 'mainentityofpageRecipe': 120,
 'makesofferPerson': 121,
 'manufacturerProduct': 122,
 'materialProduct': 123,
 'memberofPerson': 124,
 'menuRestaurant': 125,
 'modelProduct': 126,
 'mpnProduct': 127,
 'nameBook': 128,
 'nameCreativeWork': 129,
 'nameEvent': 130,
 'nameHotel': 131,
 'nameLocalBusiness': 132,
 'nameMusicAlbum': 133,
 'nameMusicRecording': 134,
 'namePlace': 135,
 'nameProduct': 136,
 'nameRecipe': 137,
 'nameRestaurant': 138,
 'nameTVEpisode': 139,
 'nationalityPerson': 140,
 'numberofpagesBook': 141,
 'numtracksMusicAlbum': 142,
 'nutritionRecipe': 143,
 'offerdetailsProduct': 144,
 'offersBook': 145,
 'offersCreativeWork': 146,
 'offersProduct': 147,
 'openinghoursLocalBusiness': 148,
 'openinghoursRestaurant': 149,
 'openinghoursspecificationLocalBusiness': 150,
 'openinghoursspecificationPlace': 151,
 'openinghoursspecificationRestaurant': 152,
 'organizerEvent': 153,
 'partofseriesTVEpisode': 154,
 'paymentacceptedLocalBusiness': 155,
 'widthProduct': 156,
 'performersEvent': 157,
 'performtimeRecipe': 158,
 'postalcodeLocalBusiness': 159,
 'preptimeRecipe': 160,
 'priceProduct': 161,
 'pricecurrencyProduct': 162,
 'pricerangeHotel': 163,
 'pricerangeLocalBusiness': 164,
 'pricerangeRestaurant': 165,
 'publisherBook': 166,
 'publisherCreativeWork': 167,
 'publisherRecipe': 168,
 'ratingvalueProduct': 169,
 'recipecategoryRecipe': 170,
 'recipecuisineRecipe': 171,
 'recipeingredientRecipe': 172,
 'recipeinstructionsRecipe': 173,
 'recipeyieldRecipe': 174,
 'releasedateProduct': 175,
 'reviewLocalBusiness': 176,
 'reviewProduct': 177,
 'reviewRecipe': 178,
 'reviewcountProduct': 179,
 'reviewsProduct': 180,
 'sameasLocalBusiness': 181,
 'sameasPerson': 182,
 'sameasPlace': 183,
 'servescuisineRestaurant': 184,
 'shop-currencyProduct': 185,
 'starratingHotel': 186,
 'streetaddressLocalBusiness': 187,
 'suitablefordietRecipe': 188,
 'telephoneHotel': 189,
 'telephoneLocalBusiness': 190,
 'telephonePerson': 191,
 'telephonePlace': 192,
 'telephoneRestaurant': 193,
 'titleProduct': 194,
 'totaltimeRecipe': 195,
 'trackMusicAlbum': 196,
 'typicalagerangeEvent': 197,
 'versionCreativeWork': 198,
 'weightPerson': 199,
 'weightProduct': 200,
 'founderLocalBusiness': 201
 #'eventattendancemodeEvent': 201,
 #'performerEvent': 202,
 #'inalbumMusicRecording': 203,
 #'eventstatusEvent': 204
 }

In [21]:
train['label'] = train.label_complete.map(labels_new)
test['label'] = test.label_complete.map(labels_new)
train.drop(['label_complete', 'name'], axis=1, inplace=True)
test.drop(['label_complete', 'name'], axis=1, inplace=True)

In [22]:
train.head()

,text,label
0,Bierwurst com Cupids Dating Tips com Ramsbury ...,136
1,price 12950 availability https schema org InS...,147
2,Food and Drink Lifestyle Dating Travel Design ...,38
3,2020-09-16T02 57 04 01 00 2020-09-10T08 33 11 ...,175
4,name pa internet-archive-age value 9 years va...,3


In [23]:
test.head()

,text,label
0,Plauen Indonesien Freiburg Iraq-Kurdistan Boch...,130
1,2022-03-27T16 30 00 01 00 2020-11-15T22 00 00 ...,68
2,name Abenteuer Arena name Paulussaal name Deu...,115
3,name Freitr umer Festival name MUNDOlogia nam...,153
4,M y l m kem Sunny Way RB1119A T M T 2 C NH K N...,136


### Baseline Simple transformer

General model info

In [24]:
# arguments
train_args = {
    'num_train_epochs': 25,
    'overwrite_output_dir': True
}

Roberta

In [ ]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=num_columns, args=train_args)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
model = ClassificationModel(
    "roberta", "/content/outputs/checkpoint-5680-epoch-4",
    args=train_args
)

In [ ]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11356 [00:00<?, ?it/s]

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 0 of 25:   0%|          | 0/1420 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 25:   0%|          | 0/1420 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model.train_model(train)

In [ ]:
model = ClassificationModel(
    "roberta", "outputs/checkpoint-1420-epoch-1"
)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/7033 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/880 [00:00<?, ?it/s]

In [ ]:
pred = [np.argmax(o) for o in model_outputs]
true = test.label.to_list()
import sklearn
sklearn.metrics.f1_score(true, pred, average='micro')

0.7333996871889663

In [ ]:
%%bash
mv '/content/outputs/' '/content/drive/My Drive/1_Studium/2_Master/3_Sem/Team_Project/Code/'

Distilbert

In [ ]:
#from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel('distilbert', 'distilbert-base-uncased', args=train_args, num_labels=num_columns) 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
model.train_model(train)

RuntimeError: ignored

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

In [ ]:
pred = [np.argmax(o) for o in model_outputs]
true = test.label.to_list()
import sklearn
sklearn.metrics.f1_score(true, pred, average='micro')

Bert

In [25]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=num_columns, args=train_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
model.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11356 [00:00<?, ?it/s]

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 0 of 25:   0%|          | 0/1420 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

In [ ]:
pred = [np.argmax(o) for o in model_outputs]
true = test.label.to_list()
import sklearn
sklearn.metrics.f1_score(true, pred, average='micro')